# Monad transformers

## Outline

* Incentive for the Monad transformers

* The MonadIO type class

* Monad Transformers
  * StateT 

  * WriterT

  * ReaderT

  * MaybeT

  * EitherT

In this lesson, we will learn about Monad transformers and how you can use them. First we show the incentive for Monad transformers where we solve a problem with the help of the MonadIO type class and explain it. Then we explain the monad transformer contept and introduce various Monad transformers. For each one we show a practical example that solves a problem. 

## Incentive for Monad transformers

In the homework of the State monad lesson we learned how you can implement the Tic-Tac-Toe game with using the State monad. 

A curious student might ask how to implement this game with the State monads by geting the input from the user instead of the random funtion.

Or in other words, is it possible to perform IO actions within a user defined State monad? The answer is yes.

With the knowledge that we have so far we can create a custom State monad that we will call `StateIO`.

The idea is that the result of the `runState` function will have to be wrapped in IO.

Below is the code that implements our costum `StateIO` state monad that returns the result in IO context.

In [ ]:
{-# LANGUAGE MultiParamTypeClasses #-}
{-# LANGUAGE FlexibleInstances #-} 

import Control.Monad.Trans
import Control.Monad (ap, liftM)
import Control.Monad.State.Class (MonadState(get, put))

newtype StateIO s a = StateIO {runStateIO :: s -> IO (a, s)}

instance Functor (StateIO s) where
    fmap = liftM

instance Applicative (StateIO s) where
    pure = return
    (<*>) = ap

instance Monad (StateIO s) where
    return a = StateIO $ \s -> return (a, s)
    n >>= k = StateIO $ \s -> do (a, s') <- runStateIO n s
                                 runStateIO (k a) s'

instance MonadState s (StateIO s) where
    get = StateIO $ \st -> return (st,st)
    put st = StateIO $ const $ return ((),st)

We see that the `StateIO` type is defined similar as the `State` type just that it returns `IO (a,s)` instead of `(a,s)`.

But if we try now and create a state IO monad and try to perform some IO action in it, the code will not work.

In [ ]:
data Choice = Empty | X | O deriving (Eq, Show)
type Board = [Choice]

getUserMove :: StateIO Board ()
getUserMove = do
    board <- get
    putStrLn "Make your move X player (1-9):" -- causes compilation error
    choice <- getLine -- also causes compilation error
    let validMove = length choice == 1 && all (`elem` "123456789") choice
    if validMove
    then do
        let newBoar = [mv |ind <- [1..9], let mv = if ind == read choice then X else Empty]
        put newBoar  
    else do
        putStrLn "Not a valid choice. Make new one." -- also causes compilation error
        getUserMove
    return ()

main :: IO ()
main = do
    let board = [Empty | boardInd <- [1..9]]
    (_, newBoard) <- runStateIO getUserMove board
    putStrLn $ "New Board is: " ++ show newBoard

main

The reason for this is how the bind operator for the `StateIO` is defined. 
```haskell
(>>=) :: StateIO s a -> (a -> StateIO s b) -> StateIO s b
```

It can not take in a non `StateIO` variable variable or a function that wouldn't return a `StateIO` variable. 

The `print` function returns only `IO ()` and not `StateIO () a`, so the bind operator can not be applied to it.

The solution to this is defining a function that takes an IO action and lifts it to a `StateIO` monad.

In [ ]:
liftIOtoStateIO :: IO a -> StateIO s a
liftIOtoStateIO io = StateIO $ \st -> do x <- io
                                         return (x, st)

Now we can write again our code and it will compile correctly.

In [ ]:
data Choice = Empty | X | O deriving (Eq, Show)
type Board = [Choice]

getUserMove :: StateIO Board ()
getUserMove = do
    board <- get
    liftIOtoStateIO $ putStrLn "Make your move X player (1-9):"
    choice <- liftIOtoStateIO getLine 
    let validMove = length choice == 1 && all (`elem` "123456789") choice
    if validMove
    then do
        let newBoar = [mv |ind <- [1..9], let mv = if ind == read choice then X else Empty]
        put newBoar  
    else do
        liftIOtoStateIO $ putStrLn "Not a valid choice. Make new one."
        getUserMove
    return ()

main :: IO ()
main = do
    let board = [Empty | boardInd <- [1..9]]
    (_, newBoard) <- runStateIO getUserMove board
    putStrLn $ "New Board is: " ++ show newBoard

main

We created now a State monad that works in IO context and found a way how to perform IO actions inside of such a monad. 

Of course you could do the same and create a State monad that returns the result in a Maybe context or as a list of tuples.

Such transformations make code more practical. For this reason Haskell provides build in solutions for them.

## The MonadIO type class

Because lifting a type wraped inside IO to another monad is very usefull, Haskell handles this with the `MonadIO` type class.

It defines the `liftIO` function that generalizes the type signature of our `liftIOtoStateIO` function.
```haskell
class Monad m => MonadIO m where
    liftIO :: IO a -> m a 
```

The `MonadIO` type class is defined in the **Control.Monad.IO.Class** module which is part of the **base** package.

So insted of defining a custom function for lifting we can import this module and create an instance of `MonadIO` for the `StateIO s` type.

In [ ]:
import Control.Monad.IO.Class

instance MonadIO (StateIO s) where
    liftIO io = StateIO $ \st -> do x <- io
                                    return (x, st)

getUserMove :: StateIO Board ()
getUserMove = do
    board <- get
    liftIO $ putStrLn "Make your move X player (1-9):"
    choice <- liftIO getLine 
    let validMove = length choice == 1 && all (`elem` "123456789") choice
    if validMove
    then do
        let newBoar = [mv |ind <- [1..9], let mv = if ind == read choice then X else Empty]
        put newBoar  
    else do
        liftIO $ putStrLn "Not a valid choice. Make new one."
        getUserMove
    return ()

## Monad transformers

We ask ourselfs now how to generalize wrapping the result of the State monad function into a monadic context.

When trying to wrap an existing monad into another monad we call the function that does this a **monad transformer**.

This can be done not just for the State and the IO Monad, but all possible monad combinations can be handled with Monad transformers.

From [Haskell wikibook](https://en.wikibooks.org/wiki/Haskell/Monad_transformers): Monad transformers are special types that allow us to roll two monads into a single one that shares the behavior of both. 

There are many transformers already defined in various modules. We will cover here the following transformers:
- State monad transformer `StateT`

- Writer monad transformer `WriterT`

- Reader monad transformer `ReaderT`

- Maybe monad transformer `MaybeT`

- Either monad transformer `EitherT`

## StateT monad transformer

The `StateT` monad transformer is provided in the **Control.Monad.Trans.State** module that comes with the **transformers** package.

As said in the State monad lesson also here you can append to the module name **.Lazy** or **.Strict** to get the apropriate version.

The `StateT` tpye is defined similar as the `State` type: 
```haskell
newtype StateT s m a = StateT { runStateT :: s -> m (a,s) }
```

The monad instance for the `StateT` type is:
```haskell
instance Monad m => Monad (StateT s m) where
    return a = StateT $ \ s -> return (a, s)
    m >>= k  = StateT $ \ s -> do
        ~(a, s') <- runStateT m s
        runStateT (k a) s'
```

Here the `~` operator in the above expression means ... TODO 

We see the difference to the basic `State` type is that the result of the `runStateT` function is wrapper in a monadic context.

Also we can now understand the official Haskell definition of the `State` monad:
```haskell
type State s = StateT s Identity
```
We see the `State` monad is defined in terms of the `StateT` monad transformer.

Here the monadic context is the `Identity` type, which is only an identity wrapper for a variable of type `a`:
```haskell
newtype Identity a = Identity { runIdentity :: a }
```

This means that we do not perform and transformation with the `StateT` monad and get the basic `State` monad in return.

When defining an Functor and Applicative instance for this type we also need to add the appropriate type constraints.
```haskell
instance (Functor m) => Functor (StateT s m) where
    fmap = liftM 

instance (Functor m, Monad m) => Applicative (StateT s m) where
    pure = return
    (<*>) = ap
```

### Helper functions

The helper functions `runStateT`, `evalStateT`, `execStateT` are defined similar as their pairs for the `State` moand.

They just return the result wraped in a monad context. The later two take in the more generalized `StateT` type:
```haskell
evalStateT :: Monad m => StateT s m a -> s -> m a
execStateT :: Monad m => StateT s m a -> s -> m s 
```

The other functions that we presented in the State monad lesson are all actually defined in terms of the `StateT` monad. 

Here are their actual type signatures:
```haskell
get :: Monad m => StateT s m s 
put :: Monad m => s -> StateT s m () 
modify :: Monad m => (s -> s) -> StateT s m ()
gets :: Monad m => (s -> a) -> StateT s m a 
```

We see they work similar as described in the State monad lesson, just that they use the `StateT` type instead of the `State` type.

### Example

Now we can re-write the initial code example with the `StateIO` monad by using the `StateT` monad transformer instead.

We parameterize the `StateT` with the `IO ()` type since we want to perform IO actions and do not return any result.

In [ ]:
import Control.Monad.Trans.State.Lazy ( StateT(runStateT), get, put )
import Control.Monad.IO.Class ( MonadIO(liftIO) )

data Choice = Empty | X | O deriving (Eq, Show)
type Board = [Choice]

getUserMove :: StateT Board IO ()
getUserMove = do
    board <- get
    liftIO $ putStrLn "Make your move X player (1-9):"
    choice <- liftIO getLine 
    let validMove = length choice == 1 && all (`elem` "123456789") choice
    if validMove
    then do
        let newBoar = [mv |ind <- [1..9], let mv = if ind == read choice then X else Empty]
        put newBoar  
    else do
        liftIO $ putStrLn "Not a valid choice. Make new one."
        getUserMove
    return ()

main :: IO ()
main = do
    let board = [Empty | boardInd <- [1..9]]
    (_, newBoard) <- runStateT getUserMove board
    putStrLn $ "New Board is: " ++ show newBoard

We see there is not need to define any custom `State` monad and make instances for it, when we use the `StateT` monad with the `IO` context.

Another benifit is we do not have to declare an `MonadIO` instance for `StateT` because it allready has it declared in the module that defines it.

In the example above we used the `IO` type as a monadic wrapper for the result of the `runStateT` function. 

But we can use any other type instead of `IO` or `Identitiy`, that has a monad instance. 

## WriterT monad transformer

The `WriterT` monad transformer is provided in the **Control.Monad.Trans.Writer** module, that comes with the **transformers** package.

As said in the Writer monad lesson also here you can append to the module name **.Lazy** or **.Strict** to get the apropriate version. 

The `WriterT` type is defined similar as the `Writer` type:
```haskell
newtype WriterT w m a = WriterT { runWriterT :: m (a, w) }
```

The monad instance for the `WriterT` type is:
```haskell
instance (Monad m) => Monad (WriterT w m) where
    return a = WriterT $ \ w -> return (a, w)
    m >>= k = WriterT $ \ w -> do
        (a, w') <- runWriterT m w
        runWriterT (k a) w'
```

We see the difference to the basic `Writer` type is also here that the result of the `runWriterT` function is wrapper in a monadic context.

We can now also understand the official Haskell definition of the `Writer` monad:
```haskell
type Writer w = WriterT w Identity
```
We see the `Writer` monad is defined in terms of the `WriterT` monad transformer.

Same as for the `State` monad, the monadic context is the `Identity` monad that we explained before.

Also here this means that we do not perform and transformation with the `WriterT` monad and get the basic `Writer` monad in return.

Similar as for the `StateT` monad when defining Functor and Applicative instances we need to add the appropriate type constraint.
```haskell
instance (Functor m) => Functor (WriterT w m) where
    fmap = liftM 

instance (Functor m, Monad m) => Applicative (WriterT w m) where
    pure = return
    (<*>) = ap
```

### Helper functions

The helper functions `runWriterT` and `execWriterT` are defined similar as their pairs for the `Writer` moand.

They just returns the result wraped in a monad context. The `execWriterT` function takes in the more generalized `WriterT` type:
```haskell
execWriterT :: Monad m => WriterT w m a -> m w  
```

The other functions that we presented in the Writer monad lesson are all actually defined in terms of the `WriterT` monad.

Here are their actual type signatures:
```haskell
tell :: Monad m => w -> WriterT w m () 
listen :: Monad m => WriterT w m a -> WriterT w m (a, w) 
pass :: Monad m => WriterT w m (a, w -> w) -> WriterT w m a 
listens :: Monad m => (w -> b) -> WriterT w m a -> WriterT w m (a, b) 
censor :: Monad m => (w -> w) -> WriterT w m a -> WriterT w m a 
```

We see they work similar as described in the Writer monad lesson, just that they use  the `WriterT` type instead of the `Writer` type.

### Example

Let us rewrite our homework example from lesson 12, where we had to write a program that prints out a directory tree structure.

We simplify the task such that only file paths of every folder and file relative to the current directory are printed out.

We can use now a `WriterT` monad transformer parameterized by `[String] IO ()`. 

In [ ]:
import System.Directory (listDirectory, doesFileExist)
import Control.Monad.Trans.Writer.CPS (WriterT, execWriterT, tell) 
import Control.Monad.IO.Class (MonadIO(liftIO))
import Control.Monad (forM_)

printDirectoryTree :: IO ()
printDirectoryTree = do 
    putStrLn "."
    structure <- execWriterT (returnStructure ".")
    mapM_ print structure

returnStructure :: FilePath -> WriterT [String] IO ()
returnStructure filePath = do
    contents <- liftIO $ listDirectory filePath
    forM_ contents $ \fileName -> do
        let newFilePath = filePath ++ "/" ++ fileName
        isFile <- liftIO $ doesFileExist newFilePath
        if isFile
        then do
            tell [newFilePath]
            return ()
        else do 
            tell [newFilePath]
            _ <- returnStructure newFilePath
            return ()

Here we note that the type signature for the `returnStructure` function from the homework solution of lesson 11:
```haskell
returnStructure :: FilePath -> IO [IO DirectoryStructure]
```
gets simplified to:
```haskell
returnStructure :: FilePath -> WriterT [String] IO ()
```
and there is no need to define a custom data type as `DirectoryStructure`.

## ReaderT monad transformer

The `ReaderT` monad transformer is provided in the **Control.Monad.Trans.Reader** module, that comes with the **transformers** package.

The `ReaderT` tpye is defined similar as the `Reader` type: 
```haskell
newtype ReaderT r m a = ReaderT { runReaderT :: r -> m a }
```

The monad instance for the `ReaderT` type is: 
```haskell
instance (Monad m) => Monad (ReaderT r m) where
    return   = lift . return
    m >>= k  = ReaderT $ \ r -> do
        a <- runReaderT m r
        runReaderT (k a) r 
```

Same as for `StateT` and `WriterT` we see the difference to the basic `Reader` type is that the result of the `runReaderT` function is wrapper in a monadic context.

Also again we can now understand the official Haskell definition of the `Reader` monad:
```haskell
type Reader r = ReaderT r Identity
```
We see the `Reader` monad is defined in terms of the `ReaderT` monad transformer.

Same as for the `State` and `Writer` monad, the monadic context is the `Identity` monad that we explained before.

Also here this means that we do not perform and transformation with the `ReaderT` monad and get the basic `Reader` monad in return.

Also similar as for the `StateT` and `WriterT` monad when defining Functor and Applicative instances we need to add the appropriate type constraint.
```haskell
instance (Functor m) => Functor (ReaderT r m) where
    fmap = liftM 

instance (Applicative m) => Applicative (ReaderT r m) where
    pure = return
    (<*>) = ap
```

### Helper functions

### Example